In [74]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error
import pyper

# Rから呼び出せるか実験

In [75]:
data_url = "http://lib.stat.cmu.edu/datasets/boston"
raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
X = pd.DataFrame(np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]]))
X.columns = ["feat1","feat2","feat3","feat4","feat5","feat6","feat7","feat8","feat9","feat10","feat11","feat12","feat13"]
y = raw_df.values[1::2, 2]

In [76]:
X

,feat1,feat2,feat3,feat4,feat5,feat6,feat7,feat8,feat9,feat10,feat11,feat12,feat13
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33
...,...,...,...,...,...,...,...,...,...,...,...,...,...
501,0.06263,0.0,11.93,0.0,0.573,6.593,69.1,2.4786,1.0,273.0,21.0,391.99,9.67
502,0.04527,0.0,11.93,0.0,0.573,6.120,76.7,2.2875,1.0,273.0,21.0,396.90,9.08
503,0.06076,0.0,11.93,0.0,0.573,6.976,91.0,2.1675,1.0,273.0,21.0,396.90,5.64
504,0.10959,0.0,11.93,0.0,0.573,6.794,89.3,2.3889,1.0,273.0,21.0,393.45,6.48


In [77]:
r = pyper.R(use_pandas='True')

In [78]:
X_TRAIN = X.iloc[:350]
Y_TRAIN = y[:350]
X_VAL = X.iloc[350:450]
Y_VAL = y[350:450]
X_TEST = X.iloc[450:]
Y_TEST = y[450:]

In [72]:
r.assign("X_TRAIN", X_TRAIN)
r.assign("Y_TRAIN", Y_TRAIN)
r.assign("X_VAL", X_VAL)
r.assign("Y_VAL", Y_VAL)
r.assign("X_TEST", X_TEST)

r.assign("NTREE", 600)
r.assign("NODESIZE", 20)

In [73]:
r("source(file='regularized_random_forest.R')")

'try({source(file=\'regularized_random_forest.R\')})\nRRF 1.9.1\nType rrfNews() to see new features/changes/bug fixes.\nError in build_rrf(X_TRAIN, Y_TRAIN, NTREE, NODESIZE) : \n  could not find function "build_rrf"\n'

In [19]:
# Rのmain関数を呼び出し，結果をresultへ代入する
r("result <- main()")
# Rのresultから値を持ってくる
result = r.get("result")

In [20]:
result[0]

array([28.10469351, 23.47314974, 35.37375526, 35.72241248, 33.35282786,
       27.03909193, 20.61212131, 18.32592008, 18.4166241 , 19.82825543,
       20.49496467, 20.39813786, 20.4780209 , 20.47020855, 19.52437808,
       20.64974374, 21.14375698, 18.78740037, 18.57784025, 19.59214645,
       15.3429975 , 17.87002475, 17.33708096, 15.81475699, 17.33374966,
       17.26420021, 18.17078149, 16.91855821, 19.4186142 , 20.75380709,
       15.60020768, 18.43412498, 16.70497554, 15.78415547, 15.95016309,
       22.06949951, 21.47847127, 22.04946936, 21.3715894 , 29.49053292,
       35.64472511, 29.75061443, 24.13768527, 23.95739135, 22.06316995,
       21.00541869, 20.77507674, 20.17444192, 19.19225527, 19.95212227,
       21.05668335, 22.41134728, 26.30300503, 21.8970274 , 20.97664711,
       34.34068188, 24.95983984, 33.27900985, 23.39026989, 21.54177185,
       20.0374956 , 20.55708822, 23.98787369, 25.53022405, 31.62190812,
       26.08097566, 21.88058867, 21.66511364, 20.12427651, 21.51

# モジュールの動作確認

In [97]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [98]:
from embedded_methods import regularized_random_forest as RRF

In [137]:
model = RRF(ntree=100, nodesize=3, coefReg=1e-5)

In [138]:
a = model.fit_pred(X_TRAIN, Y_TRAIN, X_VAL, X_TEST)

In [139]:
a[0].shape

(350,)

In [140]:
Y_TRAIN

array([24. , 21.6, 34.7, 33.4, 36.2, 28.7, 22.9, 27.1, 16.5, 18.9, 15. ,
       18.9, 21.7, 20.4, 18.2, 19.9, 23.1, 17.5, 20.2, 18.2, 13.6, 19.6,
       15.2, 14.5, 15.6, 13.9, 16.6, 14.8, 18.4, 21. , 12.7, 14.5, 13.2,
       13.1, 13.5, 18.9, 20. , 21. , 24.7, 30.8, 34.9, 26.6, 25.3, 24.7,
       21.2, 19.3, 20. , 16.6, 14.4, 19.4, 19.7, 20.5, 25. , 23.4, 18.9,
       35.4, 24.7, 31.6, 23.3, 19.6, 18.7, 16. , 22.2, 25. , 33. , 23.5,
       19.4, 22. , 17.4, 20.9, 24.2, 21.7, 22.8, 23.4, 24.1, 21.4, 20. ,
       20.8, 21.2, 20.3, 28. , 23.9, 24.8, 22.9, 23.9, 26.6, 22.5, 22.2,
       23.6, 28.7, 22.6, 22. , 22.9, 25. , 20.6, 28.4, 21.4, 38.7, 43.8,
       33.2, 27.5, 26.5, 18.6, 19.3, 20.1, 19.5, 19.5, 20.4, 19.8, 19.4,
       21.7, 22.8, 18.8, 18.7, 18.5, 18.3, 21.2, 19.2, 20.4, 19.3, 22. ,
       20.3, 20.5, 17.3, 18.8, 21.4, 15.7, 16.2, 18. , 14.3, 19.2, 19.6,
       23. , 18.4, 15.6, 18.1, 17.4, 17.1, 13.3, 17.8, 14. , 14.4, 13.4,
       15.6, 11.8, 13.8, 15.6, 14.6, 17.8, 15.4, 21

In [141]:
mean_squared_error(Y_TRAIN, a[0], squared=False)

2.69041600900808

In [142]:
mean_squared_error(Y_VAL, a[1], squared=False)

8.149178786614316

In [143]:
mean_squared_error(Y_TEST, a[2], squared=False)

3.7741736489710034

In [144]:
Y_VAL

array([22.9, 24.1, 18.6, 30.1, 18.2, 20.6, 17.8, 21.7, 22.7, 22.6, 25. ,
       19.9, 20.8, 16.8, 21.9, 27.5, 21.9, 23.1, 50. , 50. , 50. , 50. ,
       50. , 13.8, 13.8, 15. , 13.9, 13.3, 13.1, 10.2, 10.4, 10.9, 11.3,
       12.3,  8.8,  7.2, 10.5,  7.4, 10.2, 11.5, 15.1, 23.2,  9.7, 13.8,
       12.7, 13.1, 12.5,  8.5,  5. ,  6.3,  5.6,  7.2, 12.1,  8.3,  8.5,
        5. , 11.9, 27.9, 17.2, 27.5, 15. , 17.2, 17.9, 16.3,  7. ,  7.2,
        7.5, 10.4,  8.8,  8.4, 16.7, 14.2, 20.8, 13.4, 11.7,  8.3, 10.2,
       10.9, 11. ,  9.5, 14.5, 14.1, 16.1, 14.3, 11.7, 13.4,  9.6,  8.7,
        8.4, 12.8, 10.5, 17.1, 18.4, 15.4, 10.8, 11.8, 14.9, 12.6, 14.1,
       13. ])

In [145]:
a[1]

array([25.182     , 25.88066667, 23.19683333, 30.85      , 22.57126667,
       23.89183333, 17.68646364, 19.29443333, 19.59513889, 19.22092222,
       22.39841667, 19.0846    , 19.16988889, 17.59225556, 37.4365    ,
       21.71880556, 17.55797222, 16.86297222, 26.19525   , 28.77508333,
       35.43583333, 21.1175    , 20.63797222, 14.06503889, 14.06503889,
       28.85433333, 17.4055    , 20.41366667, 16.7195    , 16.37366667,
       26.61703333, 16.982     , 15.25053889, 15.20903889, 15.22790556,
       14.42337222, 14.51803889, 15.49990556, 14.76653889, 14.98520556,
       16.01788889, 17.75018889, 14.77920556, 18.57086667, 16.89988889,
       17.4295    , 17.39083333, 15.55828889, 14.93287222, 16.62898889,
       15.84778889, 16.863     , 16.77183333, 14.99703889, 15.93440556,
       15.20137222, 14.29853889, 17.75747222, 15.35915   , 22.3005    ,
       19.63775   , 17.75016667, 14.92381667, 15.34665   , 14.74170556,
       16.73766667, 21.0407    , 15.54873889, 15.82378889, 22.65

In [147]:
a[4]

array([[6.33226411e+00, 7.04743781e+02],
       [0.00000000e+00, 0.00000000e+00],
       [6.55116075e+00, 1.87013605e+03],
       [5.47838412e-01, 6.45007701e+01],
       [8.39163005e+00, 9.70167008e+02],
       [1.87702107e+01, 1.06040809e+04],
       [5.09684215e+00, 6.64772805e+02],
       [8.22079904e+00, 8.15203951e+02],
       [0.00000000e+00, 0.00000000e+00],
       [7.39244032e+00, 9.48673564e+02],
       [7.25672436e+00, 1.60039990e+03],
       [6.58058718e-01, 4.92996401e+02],
       [1.23419695e+01, 6.10440428e+03]])